In [3]:
import os
from collections import defaultdict

# 이미지 파일 개수를 세는 함수
def count_images_in_folders(base_folder):
    image_counts = {}
    for folder in ['train', 'test', 'valid']:
        images_folder = os.path.join(base_folder, folder, 'images')
        if os.path.exists(images_folder):
            # .jpg 파일 개수 세기
            image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
            image_counts[folder] = len(image_files)
        else:
            image_counts[folder] = 0  # 폴더가 없을 경우 0으로 설정
    return image_counts

# 디렉토리 경로 설정
base_directory = './merged_golfball_with_golf_club'

# 이미지 개수 세기
image_counts = count_images_in_folders(base_directory)
print("이미지 파일 개수:", image_counts)


이미지 파일 개수: {'train': 24867, 'test': 1020, 'valid': 1904}


In [4]:
import os
from collections import defaultdict

# YOLO 형식 텍스트 파일에서 첫 번째 줄이 빈 줄인지 확인하는 함수
def check_first_line_empty(base_folder):
    empty_first_line_files = []
    for folder in ['train', 'test', 'valid']:
        labels_folder = os.path.join(base_folder, folder, 'labels')
        if os.path.exists(labels_folder):
            label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]
            for label_file in label_files:
                label_path = os.path.join(labels_folder, label_file)
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    # 첫 번째 줄이 비어있는지 확인
                    if len(lines) > 0 and not lines[0].strip():
                        empty_first_line_files.append(label_file)  # 빈 줄인 파일 추가
    return len(empty_first_line_files), empty_first_line_files

# YOLO 형식 텍스트 파일에서 빈 줄 또는 잘못된 형식 확인 함수
def check_label_file_format(base_folder):
    issues = []
    for folder in ['train', 'test', 'valid']:
        labels_folder = os.path.join(base_folder, folder, 'labels')
        if os.path.exists(labels_folder):
            label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]
            for label_file in label_files:
                label_path = os.path.join(labels_folder, label_file)
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    for line_number, line in enumerate(lines):
                        stripped_line = line.strip()
                        if not stripped_line:
                            # 빈 줄
                            issues.append((label_file, line_number + 1, '빈 줄'))
                        elif len(stripped_line.split()) < 5:
                            # 불완전한 데이터 (클래스 번호 + 좌표값 부족)
                            issues.append((label_file, line_number + 1, '불완전한 데이터'))
                        else:
                            # 추가적인 형식 검증 (숫자가 아닌 값이 포함된 경우)
                            parts = stripped_line.split()
                            try:
                                class_id = int(parts[0])  # 클래스 ID가 정수인지 확인
                                coords = list(map(float, parts[1:]))  # 좌표값이 실수인지 확인
                                if len(coords) != 4:  # 좌표값이 정확히 4개인지 확인
                                    issues.append((label_file, line_number + 1, '좌표값 개수 오류'))
                            except ValueError:
                                issues.append((label_file, line_number + 1, '숫자 형식 오류'))
    return issues

# 디렉토리 경로 설정
base_directory = './merged_golfball_with_golf_club'

# 첫 번째 줄이 빈 줄인지 확인
empty_first_line_count, empty_first_line_files = check_first_line_empty(base_directory)
print(f"첫 번째 줄이 빈 줄인 파일 수: {empty_first_line_count}")
if empty_first_line_files:
    print("첫 번째 줄이 빈 줄인 파일 목록:")
    for file in empty_first_line_files:
        print(file)

# 라벨 파일 포맷 확인
label_issues = check_label_file_format(base_directory)

# 문제 유형별 카운트
issue_counts = defaultdict(int)
if label_issues:
    print("\n라벨 파일에 문제가 있습니다:")
    for issue in label_issues:
        print(f"파일: {issue[0]}, 줄 번호: {issue[1]}, 문제: {issue[2]}")
        issue_counts[issue[2]] += 1

print("\n문제 유형별 개수:", dict(issue_counts))


첫 번째 줄이 빈 줄인 파일 수: 288
첫 번째 줄이 빈 줄인 파일 목록:
0112_jpg.rf.5f74f410bcec9d734c4744a36a31135d.txt
0385_jpg.rf.dc5da464c1c39a54929ef96afcadeb71.txt
040_jpg.rf.bd0b9f204f79854bbcf6a46a6c496159.txt
11_340_jpg.rf.3819e795997cec533128f5b526a49306.txt
14_303_jpg.rf.2349c653b882e2bd473ff7239e2b64f3.txt
1_174_jpg.rf.cf6dbce54de82a3f567857778e6c45f0.txt
3_174_jpg.rf.b720516d6c4a6f0ad9930e0fe34cc5de.txt
7b12db588ff57671_jpg.rf.1e914b559f10864abcff4cc720e6044f.txt
7b12db588ff57671_jpg.rf.c289079f680f5f10eb3e6b70c03f0b40.txt
IMG_0246-151_jpg.rf.ecdc125e9c50bcb640311c38f919b02f.txt
IMG_0246-151_jpg.rf.f43f86ae8f1c1fec335b69e2093e5003.txt
IMG_0430-032_jpg.rf.addfb8e0652487f5a90c1c8f391fa26a.txt
IMG_0449-104_jpg.rf.2401115b3905e6e3b9fd8439c0a8a409.txt
IMG_1819-028_jpg.rf.6f75e1e4e094ba080edfa2a3b84bde8f.txt
IMG_1822-009_jpg.rf.5b4b98f908aaaf32523adee9242a64e9.txt
IMG_1822-009_jpg.rf.b77a075323afeb1f89daa91e8ab10685.txt
Image139_png_jpg.rf.eb14c57e0260927a50a334b3abfb5bc2.txt
Image155_png_jpg.rf.0e40089f218

In [5]:
import os

# 비어있는 라벨 파일을 찾아서 해당하는 이미지 파일도 삭제하는 함수
def delete_empty_label_files(base_folder):
    deleted_files = []  # 삭제된 파일 목록 저장
    for folder in ['train', 'test', 'valid']:
        labels_folder = os.path.join(base_folder, folder, 'labels')
        images_folder = os.path.join(base_folder, folder, 'images')
        
        if os.path.exists(labels_folder) and os.path.exists(images_folder):
            label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]
            for label_file in label_files:
                label_path = os.path.join(labels_folder, label_file)
                
                # 라벨 파일이 비어 있는지 체크
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    # 라벨 파일이 비어있거나 공백으로만 이루어진 경우
                    if not lines or all(line.strip() == '' for line in lines):
                        # 이미지 파일 경로 생성
                        image_file = label_file.replace('.txt', '.jpg')
                        image_path = os.path.join(images_folder, image_file)
                        
                        # 이미지 파일 삭제
                        if os.path.exists(image_path):
                            os.remove(image_path)
                            deleted_files.append(image_path)
                            print(f'이미지 파일 삭제: {image_path}')
                        
                        # 라벨 파일 삭제
                        os.remove(label_path)
                        deleted_files.append(label_path)
                        print(f'라벨 파일 삭제: {label_path}')
    
    return deleted_files

# 디렉토리 경로 설정
base_directory = './merged_golfball_with_golf_club'

# 비어있는 라벨 파일 삭제 및 이미지 파일 삭제
deleted_files = delete_empty_label_files(base_directory)
print("\n삭제된 파일 목록:")
for file in deleted_files:
    print(file)


이미지 파일 삭제: ./merged_golfball_with_golf_club/train/images/0112_jpg.rf.5f74f410bcec9d734c4744a36a31135d.jpg
라벨 파일 삭제: ./merged_golfball_with_golf_club/train/labels/0112_jpg.rf.5f74f410bcec9d734c4744a36a31135d.txt
이미지 파일 삭제: ./merged_golfball_with_golf_club/train/images/0385_jpg.rf.dc5da464c1c39a54929ef96afcadeb71.jpg
라벨 파일 삭제: ./merged_golfball_with_golf_club/train/labels/0385_jpg.rf.dc5da464c1c39a54929ef96afcadeb71.txt
이미지 파일 삭제: ./merged_golfball_with_golf_club/train/images/040_jpg.rf.bd0b9f204f79854bbcf6a46a6c496159.jpg
라벨 파일 삭제: ./merged_golfball_with_golf_club/train/labels/040_jpg.rf.bd0b9f204f79854bbcf6a46a6c496159.txt
이미지 파일 삭제: ./merged_golfball_with_golf_club/train/images/11_340_jpg.rf.3819e795997cec533128f5b526a49306.jpg
라벨 파일 삭제: ./merged_golfball_with_golf_club/train/labels/11_340_jpg.rf.3819e795997cec533128f5b526a49306.txt
이미지 파일 삭제: ./merged_golfball_with_golf_club/train/images/14_303_jpg.rf.2349c653b882e2bd473ff7239e2b64f3.jpg
라벨 파일 삭제: ./merged_golfball_with_golf_club/tra

In [6]:
import os
from collections import defaultdict

# 이미지 파일 개수를 세는 함수
def count_images_in_folders(base_folder):
    image_counts = {}
    for folder in ['train', 'test', 'valid']:
        images_folder = os.path.join(base_folder, folder, 'images')
        if os.path.exists(images_folder):
            # .jpg 파일 개수 세기
            image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
            image_counts[folder] = len(image_files)
        else:
            image_counts[folder] = 0  # 폴더가 없을 경우 0으로 설정
    return image_counts

# 디렉토리 경로 설정
base_directory = './merged_golfball_with_golf_club'

# 이미지 개수 세기
image_counts = count_images_in_folders(base_directory)
print("이미지 파일 개수:", image_counts)


이미지 파일 개수: {'train': 24579, 'test': 1020, 'valid': 1904}


In [8]:
import os
from collections import defaultdict

# YOLO 형식 텍스트 파일에서 클래스 번호 개수를 세는 함수
def count_classes_in_labels(base_folder):
    class_counts = defaultdict(int)
    for folder in ['train', 'test', 'valid']:
        labels_folder = os.path.join(base_folder, folder, 'labels')
        if os.path.exists(labels_folder):
            label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]
            for label_file in label_files:
                label_path = os.path.join(labels_folder, label_file)
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    for line in lines:
                        stripped_line = line.strip()
                        # 빈 줄이나 잘못된 형식을 건너뜀
                        if not stripped_line or len(stripped_line.split()) < 5:
                            continue
                        try:
                            class_id = int(stripped_line.split()[0])
                            class_counts[class_id] += 1
                        except ValueError:
                            continue
    return dict(class_counts)

# 디렉토리 경로 설정
base_directory = './merged_golfball_with_golf_club'

# 클래스별 개수 세기
class_counts = count_classes_in_labels(base_directory)

print("\n클래스별 개수:", class_counts)



클래스별 개수: {0: 24077, 1: 21725, 2: 12193}


In [ ]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./merged_golfball_with_golf_club/data.yaml',  # YAML 파일 경로
    epochs=500, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golfball_with_golfclub_merged_500epoch_16batch_10patience',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 x활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

New https://pypi.org/project/ultralytics/8.3.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./merged_golfball_with_golf_club/data.yaml, epochs=500, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_golfball_with_golfclub_merged_500epoch_16batch_10patience2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, 

train: Scanning /root/YOLO_train/merged_golfball_with_golf_club/train/labels... 24579 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24579/24579 [00:24<00:00, 1009.70it/s]


train: New cache created: /root/YOLO_train/merged_golfball_with_golf_club/train/labels.cache


val: Scanning /root/YOLO_train/merged_golfball_with_golf_club/valid/labels... 1904 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1904/1904 [00:01<00:00, 953.46it/s] 

val: New cache created: /root/YOLO_train/merged_golfball_with_golf_club/valid/labels.cache


Plotting labels to runs/detect/yolov8n_golfball_with_golfclub_merged_500epoch_16batch_10patience2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/yolov8n_golfball_with_golfclub_merged_500epoch_16batch_10patience2
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/500      2.49G      2.013      3.868      1.399         41        640:  62%|██████▏   | 953/1537 [01:09<00:36, 16.03it/s]